In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "COVID-19"
cohort = "GSE227080"

# Input paths
in_trait_dir = "../../input/GEO/COVID-19"
in_cohort_dir = "../../input/GEO/COVID-19/GSE227080"

# Output paths
out_data_file = "../../output/preprocess/COVID-19/GSE227080.csv"
out_gene_data_file = "../../output/preprocess/COVID-19/gene_data/GSE227080.csv"
out_clinical_data_file = "../../output/preprocess/COVID-19/clinical_data/GSE227080.csv"
json_path = "../../output/preprocess/COVID-19/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Early differentially expressed immunological genes   in mild and severe COVID-19"
!Series_summary	"We retrospectively analysed the expression of 579 immunological genes in 60 COVID-19 subjects (SARS +ve) and 59 COVID-negative (SARS -ve) subjects using the NanoString nCounter (Immunology panel), a technology based on multiplexed single-molecule counting. Biobanked Human peripheral blood mononuclear cells (PBMCs) samples underwent Nucleic Acid extraction and digital detection of mRNA to evaluate changes in antiviral gene expression between SARS -ve controls and patients with mild (SARS +ve Mild) and moderate/severe (SARS +ve Mod/Sev) disease."
!Series_overall_design	"119 samples (60 SARS-CoV-2 positive / 59 SARS-CoV-2 negative)"
Sample Characteristics Dictionary:
{0: ['gender: F', 'gender: M'], 1: ['age: 38', 'age: 66', 'age: 21', 'age: 29', 'age: 73', 'age: 35', 'age: 48', 'age: 70', 'age: 69', 'age: 31', 'age: 72', 'age: 41', 'age: 85', 'age: 79',

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import numpy as np
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any, List

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains expression data of 579 immunological genes
# This is gene expression data, not miRNA or methylation data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 2  # "severity" indicates COVID-19 severity status
age_row = 1    # Age information is available
gender_row = 0  # Gender information is available

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert COVID-19 severity trait to binary (0 for negative/mild, 1 for moderate/severe)"""
    if not value or ':' not in value:
        return None
    severity = value.split(':', 1)[1].strip().upper()
    if severity == 'NEG':  # COVID-negative
        return 0
    elif severity == 'MILD':  # Mild COVID
        return 0
    elif severity == 'MOD_SEV':  # Moderate/Severe COVID
        return 1
    return None

def convert_age(value: str) -> float:
    """Convert age string to float"""
    if not value or ':' not in value:
        return None
    age_str = value.split(':', 1)[1].strip()
    try:
        return float(age_str)
    except ValueError:
        return None

def convert_gender(value: str) -> int:
    """Convert gender string to binary (0 for female, 1 for male)"""
    if not value or ':' not in value:
        return None
    gender = value.split(':', 1)[1].strip().upper()
    if gender == 'F':
        return 0
    elif gender == 'M':
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering and saving metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # This simulates clinical data based on the provided sample characteristics
    sample_characteristics = {
        0: ['gender: F', 'gender: M'], 
        1: ['age: 38', 'age: 66', 'age: 21', 'age: 29', 'age: 73', 'age: 35', 'age: 48', 'age: 70', 'age: 69', 'age: 31', 'age: 72', 'age: 41', 'age: 85', 'age: 79', 'age: 46', 'age: 57', 'age: 87', 'age: 52', 'age: 36', 'age: 77', 'age: 82', 'age: 89', 'age: 94', 'age: 54', 'age: 23', 'age: 61', 'age: 75', 'age: 25', 'age: 43', 'age: 24'], 
        2: ['severity: MILD', 'severity: MOD_SEV', 'severity: NEG']
    }
    
    # We don't have the actual samples yet, so we'll create placeholder sample IDs
    # The actual clinical_data processing will happen later when we have the full data
    print("Processing clinical data with sample characteristics")
    print(f"Trait row: {trait_row}, Age row: {age_row}, Gender row: {gender_row}")
    
    # Since we're just doing initial validation at this point, we'll note that clinical data
    # is available but requires further processing in subsequent steps
    print(f"Clinical data is available for processing. Trait data ({trait_row}): {sample_characteristics[trait_row]}")
    print(f"Age data ({age_row}): {sample_characteristics[age_row][:5]}...")
    print(f"Gender data ({gender_row}): {sample_characteristics[gender_row]}")
    
    # Note: We need the actual GEO data matrix with sample IDs to properly extract clinical features
    # This will be done in a later step when we have access to the complete dataset
    print("Clinical feature extraction will be completed when the full dataset is available.")


Processing clinical data with sample characteristics
Trait row: 2, Age row: 1, Gender row: 0
Clinical data is available for processing. Trait data (2): ['severity: MILD', 'severity: MOD_SEV', 'severity: NEG']
Age data (1): ['age: 38', 'age: 66', 'age: 21', 'age: 29', 'age: 73']...
Gender data (0): ['gender: F', 'gender: M']
Clinical feature extraction will be completed when the full dataset is available.


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/COVID-19/GSE227080/GSE227080_family.soft.gz
Matrix file: ../../input/GEO/COVID-19/GSE227080/GSE227080_series_matrix.txt.gz
Found the matrix table marker at line 63
Gene data shape: (579, 119)
First 20 gene/probe identifiers:
['ABCB1', 'ABL1', 'ADA', 'AHR', 'AICDA', 'AIRE', 'APP', 'ARG1', 'ARG2', 'ARHGDIB', 'ATG10', 'ATG12', 'ATG16L1', 'ATG5', 'ATG7', 'ATM', 'B2M', 'B3GAT1', 'BATF', 'BATF3']


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers appear to be standard human gene symbols (like ABCB1, ABL1, ADA, etc.)
# These are official gene symbols that don't require mapping to other identifiers

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the previously identified feature rows
# Use the clinical data from Step 1 and the row identifiers from Step 2
clinical_features = geo_select_clinical_features(
    clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Create directory for clinical data output
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)

# Save the clinical features
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")

# Preview the clinical features
clinical_features_preview = preview_df(clinical_features.T)
print("Clinical features preview:")
print(clinical_features_preview)

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=True,  # We have trait data as identified in Step 2
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data for COVID-19 severity analysis."
)

# 7. Save the linked data if it's usable
if is_usable:
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save the linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved due to quality issues.")

Gene data shape after normalization: (561, 119)
Normalized gene data saved to ../../output/preprocess/COVID-19/gene_data/GSE227080.csv
Clinical features saved to ../../output/preprocess/COVID-19/clinical_data/GSE227080.csv
Clinical features preview:
{'COVID-19': [0.0, 1.0, 1.0, 0.0, 1.0], 'Age': [38.0, 66.0, 21.0, 29.0, 73.0], 'Gender': [0.0, 1.0, 1.0, 1.0, 1.0]}
Linked data shape: (119, 564)
Linked data shape after handling missing values: (119, 564)


For the feature 'COVID-19', the least common label is '1.0' with 34 occurrences. This represents 28.57% of the dataset.
The distribution of the feature 'COVID-19' in this dataset is fine.



Quartiles for 'Age':
  25%: 47.0
  50% (Median): 63.0
  75%: 75.0
Min: 21.0
Max: 94.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 44 occurrences. This represents 36.97% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Linked data saved to ../../output/preprocess/COVID-19/GSE227080.csv
